In [3]:
import pandas as pd
import numpy as np

%store -r df 

pd.set_option('display.width', 200)  # Set a large width
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping


print(df)

                                   open          high         low       close       volume  trade_count          vwap
symbol  timestamp                                                                                                    
BTC/USD 2022-03-09 06:00:00  41521.5300  42590.080000  38900.0000  39219.0800  1364.981479      43231.0  41404.787303
        2022-03-10 06:00:00  39235.6400  40301.730000  38200.2000  38792.2300  1181.691366      35917.0  39139.215264
        2022-03-11 06:00:00  38784.6100  40250.000000  38316.0000  39167.7500   831.507763      30547.0  39192.837122
        2022-03-12 06:00:00  39172.7600  39439.780000  38666.6700  39109.3600   301.549334      16474.0  39079.401184
        2022-03-13 06:00:00  39126.4600  39198.040000  37572.5100  38520.9900   576.983541      25701.0  38409.369059
...                                 ...           ...         ...         ...          ...          ...           ...
ETH/USD 2025-03-03 06:00:00   2449.5135   2449.513500   

## VWAP Crossover Trade Signals

In [4]:
def generate_vwap_signals(df):
    # Reset index to work with symbol and timestamp as columns
    df = df.reset_index()

    # Create signals dataframe
    signals = pd.DataFrame(index=df.index)
    signals['symbol'] = df["symbol"]
    signals['timestamp'] = df['timestamp']

    # Calculate Previous close and VWAP for crossover detection
    signals['close'] = df['close']
    signals['vwap'] = df['vwap']
    signals['prev_close'] = df['close'].shift(1)
    signals['prev_vwap'] = df['vwap'].shift(1)

    # Generate signals 
    # Buy: previous close below previous VWAP and current close above current VWAP
    # Sell: previous close above previous VWAP and current close below current VWAP
    signals['signal'] = 0 # Default no signal
    signals.loc[(signals['prev_close'] < signals['prev_vwap']) &
                (signals['close'] > signals['vwap']), 'signal'] = 1 #Buy
    signals.loc[(signals['prev_close'] > signals['prev_vwap']) &
                (signals['close'] < signals['vwap']), 'signal'] = -1 #Sell
    
    # Group by symbol to handle each symbol separately
    signals['symbol_change'] = signals['symbol'] != signals['symbol'].shift(1)
    signals.loc[signals['symbol_change'], ['prev_close', 'prev_vwap']] = np.nan

    # Clean up signals at symbol boundaries
    signals.loc[signals['symbol_change'], 'signal'] = 0

    # Drop temporary columns
    signals = signals.drop(['prev_close', 'prev_vwap', 'symbol_change'], axis=1)

    # Set index back to symbol and timestamp
    signals = signals.set_index(['symbol', 'timestamp'])

    return signals

# Generate signals
signal_df = generate_vwap_signals(df)

# Merge signals with original dataframe if needed 
result_df = df.join(signal_df['signal'])

# Display sample of results
print("\nSample of result with signals:")
print(result_df[['close', 'vwap', 'signal']].head(10))

# Count signals by symbol
print("\nSignal counts by symbol:")
print(result_df.groupby('symbol')['signal'].value_counts())




Sample of result with signals:
                                close          vwap  signal
symbol  timestamp                                          
BTC/USD 2022-03-09 06:00:00  39219.08  41404.787303       0
        2022-03-10 06:00:00  38792.23  39139.215264       0
        2022-03-11 06:00:00  39167.75  39192.837122       0
        2022-03-12 06:00:00  39109.36  39079.401184       1
        2022-03-13 06:00:00  38520.99  38409.369059       0
        2022-03-14 05:00:00  38783.83  39072.780504      -1
        2022-03-15 05:00:00  39124.91  39671.189538       0
        2022-03-16 05:00:00  41359.77  40457.234853       1
        2022-03-17 05:00:00  40642.32  40814.985379      -1
        2022-03-18 05:00:00  41825.27  41307.749247       1

Signal counts by symbol:
symbol   signal
BTC/USD   0        504
          1        296
         -1        295
ETH/USD   0        507
         -1        294
          1        294
Name: count, dtype: int64


## Integrate Supertrend for signal confirmation